In [1]:
import re
import urllib3
import requests
import pandas as pd
import matplotlib.pyplot as plt

from requests.auth import HTTPBasicAuth

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
urllib3.disable_warnings(UserWarning)

plt.rcParams.update({'font.size': 15})
plt.figure(figsize=(20, 20))

pd.set_option('precision', 0)
pd.options.display.float_format = '{:,.0f}'.format

In [2]:
class CorpusSizeUtils(object):

    def __init__(self):
        pass

    @staticmethod
    def get_index_size(site='https://nlp.ncsoft.com:9200'):
        """인덱스 크기를 조회한다."""
        url = '{site}/_cat/indices?v&s=index&h=index,docs.count'.format(
            site=site)

        resp = requests.get(
            url=url,
            auth=HTTPBasicAuth('elastic', 'nlplab'),
            verify=False,
        )

        alias = {
            'sports-reply': 'sports_reply'
        }

        index = {'total': 0}
        for l in resp.text.split('\n')[1:]:
            if l == '' or l[0] == '.':
                continue

            for src in alias:
                l = l.replace(src, alias[src])

            # 사이즈 분리
            i_name, count = re.sub(
                r'crawler-(.+?)\s+(\d+)$',
                r'\g<1>\t\g<2>',
                l,
            ).split('\t', maxsplit=1)

            if count.isdecimal():
                count = int(count)

            if count == 0:
                continue

            # 인덱스명 파싱: site - category - year
            t = i_name.split('-')

            year = ''

            # 년도
            if t[-1].isdecimal():
                year = t[-1]
                t.pop()

            site = t[0]
            t = t[1:]

            category = '_'.join(t)

            index['total'] += count

            if site not in index:
                index[site] = {}

            if category not in index[site]:
                index[site][category] = {}

            if year != '':
                if year not in index[site][category]:
                    index[site][category][year] = 0

                index[site][category][year] += count
            else:
                category = '-'.join([site, category])

                if 'etc' not in index:
                    index['etc'] = {}

                if category not in index['etc']:
                    index['etc'][category] = 0

                index['etc'][category] += count

        return index

    @staticmethod
    def add_sum_column(data):
        """합계 컬럼을 넣는다."""
        df = pd.DataFrame(data).fillna(0)

        df.index.name = 'year'

        df['합계'] = df.sum(axis=1)
        df.loc['합계'] = df.sum(axis=0)

        return df

    def get_index_size_df(self):
        """ """
        index_size = self.get_index_size()
        
        df = {'total': index_size['total']}
        for k in index_size:
            if k == 'total':
                continue

            if k == 'etc':
                df[k] = pd.DataFrame({'count': index_size[k]}).fillna(0)
                df[k] = df[k].astype('float')
            else:
                df[k] = pd.DataFrame(index_size[k]).fillna(0)
                df[k] = df[k].astype('float')

                df[k].index.name = 'year'
                df[k].sort_index(inplace=True)

        return df

    def get_report(self, size_df):
        """ """
        from IPython.display import display, HTML

        html = '<h1>전체 수량: {total:,.0f}</h1>'.format(total=size_df['total'])
        display(HTML(html))

        for k in size_df:
            if k == 'total':
                continue

            df = self.add_sum_column(size_df[k])

            html = '<style>.dataframe td { text-align: right; } .dataframe th { text-align: center; }</style>'
            html += '<h1>{title}: {total:,.0f}</h1>'.format(
                title=k,
                total=df.at['합계', '합계'],
            )
            html += df.to_html(index_names=False)
            display(HTML(html))

        return

    def get_report2(self, size_df):
        """ """
        from IPython.display import display, HTML

        for k in size_df:
            if k == 'total':
                continue

            size_df[k].plot(kind='barh', figsize=(15, 10))

        return

In [3]:
utils = CorpusSizeUtils()
size_df = utils.get_index_size_df()

size_df.keys(), '{:,}'.format(size_df['total'])

(dict_keys(['total', 'bbs', 'daum', 'jisikman', 'nate', 'naver', 'etc']),
 '180,706,471')

In [4]:
utils.add_sum_column(size_df['naver']).T.to_excel('naver.xlsx')

In [5]:
utils.get_report(size_df=size_df)

,inven,lineagem,mlbpark_bullpen,mlbpark_kbo,mlbpark_mlb,ruliweb,합계
2000,0,0,0,0,0,2,2
2003,0,0,0,0,0,"10,260","10,260"
2004,0,0,0,0,0,"24,385","24,385"
2005,0,0,0,0,0,"17,605","17,605"
2006,0,0,0,0,0,"13,311","13,311"
2007,0,0,0,0,0,"10,474","10,474"
2008,0,0,0,0,0,"12,139","12,139"
2009,0,0,0,0,0,"9,172","9,172"
2010,0,0,0,0,0,"8,953","8,953"
2011,0,0,0,"103,250",0,"5,679","108,929"


,culture,economy,international,it,opinion,politics,society,sports,합계
2001,385,0,"10,842",350,0,"4,705",0,0,"16,282"
2002,"3,343","26,282","31,897","9,903",0,"14,360","16,787",0,"102,572"
2003,"1,551","48,715","49,197","22,321",0,"23,507","69,363",0,"214,654"
2004,"23,917","53,853","42,164","20,034",0,"32,794","224,841",0,"397,603"
2005,"41,096","57,435","48,635","37,207",0,"53,125","452,793",0,"690,291"
2006,"44,075","57,170","53,478","45,037",0,"54,233","724,673",0,"978,666"
2007,"52,446","428,927","59,860","45,828",0,"55,350","613,518",0,"1,255,929"
2008,"51,968","625,506","59,276","47,121",0,"56,266","669,768",0,"1,509,905"
2009,"51,967","840,584","59,142","50,501",0,"57,451","832,287",0,"1,891,932"
2010,"48,722","845,377","57,702","52,975",0,"58,469","729,388",0,"1,792,633"


,,합계
2007,"473,479","473,479"
2008,"3,940,787","3,940,787"
2009,"4,372,347","4,372,347"
2010,"4,153,137","4,153,137"
2011,"2,893,529","2,893,529"
2012,"1,172,001","1,172,001"
2013,"741,273","741,273"
2014,"1,192,179","1,192,179"
2015,"761,653","761,653"
2016,"143,234","143,234"


,economy,entertainment,international,it,politics,society,sports,합계
2004,0,0,0,0,"34,589",0,0,"34,589"
2005,"298,506","43,168","61,844","47,643","37,631","225,993","26,108","740,893"
2006,"387,091","59,262","88,359","68,991","43,561","285,968","37,058","970,290"
2007,"470,275","151,262","75,044","89,216","74,992","493,558","73,841","1,428,188"
2008,"557,529","173,423","95,018","118,891","103,464","635,651","90,524","1,774,500"
2009,"823,297","233,038","128,252","82,539","107,824","717,095","130,461","2,222,506"
2010,"1,024,136","362,637","134,752","118,429","122,867","776,296","127,555","2,666,672"
2011,"1,285,242","515,059","180,285","139,724","168,302","917,503","172,418","3,378,533"
2012,"1,460,373","576,967","157,940","162,287","146,091","1,165,287","231,097","3,900,042"
2013,"1,517,049","675,518","190,841","200,299","185,061","1,516,499","291,540","4,576,807"


,economy,international,international_error,it,kin_detail,living,opinion,politics,society,sports,sports_reply,terms_detail,tv,tv_error,합계
1997,29,25,0,50,0,48,0,0,0,0,0,0,0,0,152
1998,"28,725","12,392",0,"3,526",0,"7,345",711,"24,480","33,903",0,0,0,0,0,"111,082"
1999,"34,039","15,824",0,"4,242",0,"8,867",562,"28,806","64,972",0,0,0,0,0,"157,312"
2000,"132,507","29,147",0,"11,717",0,"9,724","18,696","36,357","81,462",0,0,0,0,0,"319,610"
2001,"169,841","46,820",0,"15,343",0,"17,639","26,994","44,832","102,387",0,0,0,0,0,"423,856"
2002,"176,123","55,857",0,"58,657",0,"13,760","35,243","62,881","118,225",0,0,0,0,0,"520,746"
2003,"580,971","86,765",0,"110,666",0,"40,489","49,044","105,687","164,981",0,0,0,"34,259",0,"1,172,862"
2004,"727,553","89,894",0,"112,423",0,"107,851","41,435","157,740","191,538","24,853",0,0,"69,980",0,"1,523,267"
2005,"696,190","108,077",0,"140,354",0,"135,910","52,752","165,361","197,313","42,880",0,0,"99,288",0,"1,638,125"
2006,"733,966","128,364",0,"142,605",0,"182,411","50,430","167,803","196,038","63,938",0,0,"141,083",0,"1,806,638"


,count,합계
naver-international_error,2,2
naver-kin_detail,"3,342,508","3,342,508"
naver-terms_detail,"1,363,306","1,363,306"
naver-tv_error,4,4
합계,"4,705,820","4,705,820"
